In [1]:
import pandas as pd
import numpy as np
import time
import os
import json
from multiprocessing import Process, Manager, Value, Array, managers
from numpy import genfromtxt
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from multiprocessing import Pool
import collections
import itertools
from itertools import combinations
# from more_itertools import unique_everseen
from Bio.Seq import Seq
# from Bio import pairwise2

import os
import time
import pandas as pd
import numpy as np
from random import sample

import torch
import torch.nn as nn
import torch.nn.functional as F

from Bio import SeqIO
from multiprocessing import Process

import matplotlib.pyplot as plt

# Calculation

kaligned_path = 'E_coli_orthoslc/kalign/'

def get_distance(str1, str2):
    d = 0
    l = len(str1)
    
    for c in range(l):
        if str1[c] != str2[c]:
            d = d + 1
    d = d
    
    return d

def get_contribution(in_fasta_path, rt_dict):
    in_fasta = SeqIO.to_dict(SeqIO.parse(kaligned_path + in_fasta_path,
                                         'fasta')
                            )
    
    rep_d = {}
    
    for keys in in_fasta.keys():
        
        seq = str(in_fasta[keys].seq)
        
        if seq in rep_d.keys():
            rep_d[seq].append(keys)
        else:
            rep_d[seq] = [keys]
            
    key_lst = list(rep_d.keys())
    
    list_seqs = combinations(key_lst, 2)
    
    S_contribution = 0
    
    for pair in list_seqs:
#         a = len(pair[0])
#         b = len(pair[1])
#         if a < b:
#             bi_zhi = a/b
#         else:
#             bi_zhi = b/a
    
        a = len(rep_d[pair[0]]) # number of identical seqs of this representation in that cluster
        b = len(rep_d[pair[1]])
        
#         S_contribution = bi_zhi*get_distance(pair[0], pair[1])*(a + b)**1/len(in_fasta.keys()
#                                                                                        )
#                                                                                  )
        S_contribution = S_contribution + get_distance(pair[0], pair[1])*(a + b)
    
    rt_dict[in_fasta_path.replace('.fasta', '')] = [S_contribution, len(rep_d.keys()), len(pair[0])]
    
def divide_chunks(l, n):

    # looping till length l
    for i in range(0, len(l), n):
        yield l[i:i + n] 

start_time_external = time.time()
print(time.asctime())

if __name__ == '__main__':
    procc_num = 16
    
    mission_lst = list(divide_chunks(os.listdir(kaligned_path),
                                     procc_num
                                    )
                      )
    
    manager = Manager()
    return_dict = manager.dict()
    
    for submissions in mission_lst:
    
        jobs = []

        for x in range(0, len(submissions)):
            p = Process(target = get_contribution,
                        args = (submissions[x], 
                                return_dict
                               )
                       )
            p.start()
            jobs.append(p)


        for z in jobs:
            z.join()
        
end_time_external = time.time()
print("final time usage " + str(end_time_external - start_time_external))
print(time.asctime())

df = pd.DataFrame.from_dict(return_dict, orient='index')
df = df.sort_values(by = [0], ascending = False)
df

df.to_csv('E_coli_orthoslc/cluster_contribution.csv',
          header = False)

# KO

In [2]:
df = pd.read_csv('E_coli_orthoslc/cluster_contribution.csv', 
                 header = None)

# with open('/data/docker_qiime2_share_container_D/to_github/pt/CoreDL/pt/strains_id.json', 'r') as f_read:
#     read = json.load(f_read)
# print(len(read))

with open('E_coli_orthoslc/Step1_pre_res.txt', 'r') as f_read:
    rls = f_read.readlines()
    
    
d = {}
for x in rls:
    l_ = x.rstrip('\n').split('\t')
    d[l_[0]] = l_[1]
    
f_read.close()
    
# inv_map = {str(v): k for k, v in d.items()}
inv_map = d

In [3]:
df.iloc[0: 5, ]

,0,1,2,3
0,11002-12335,75319401,328,4358
1,10761-11925,27280561,238,2853
2,10416-10370,20438341,222,2818
3,10725-12709,19374959,202,2044
4,10938-12546,19092397,282,2769


In [4]:
a = list(df[0])

# for x in a[715: ]:
for x in a[650: ]:
    a.remove(x)

In [5]:
len(a)

650

In [6]:
to_skip = a
len(to_skip)

650

In [7]:
print(time.asctime())
kaligned_path = '/data/docker_qiime2_share_container_D/to_github/pt/CoreDL/pt/E_coli_orthoslc/kalign/'
# final_core_genome_path = "/data/docker_qiime2_share_container_D/to_github/pt/CoreDL/pt/E_coli_orthoslc/the_core_genome_1127.fasta"

# to_write = []



seq_coll = {}

ini_seq = SeqIO.to_dict(SeqIO.parse(kaligned_path + "10004-14472.fasta", 
                                    "fasta")
                       )
for x in ini_seq:
    seq_coll[inv_map[x[0: 5]]] = ""

for x in os.listdir(kaligned_path):
#     print(x)
    x_fasta = SeqIO.to_dict(SeqIO.parse(kaligned_path + x, 
                                        "fasta")
                           )
    
    if x.replace('.fasta', '') in to_skip:
        for y in x_fasta:
            seq_coll[inv_map[y[0: 5]
                            ]
                    ] = seq_coll[inv_map[y[0: 5]
                                        ]
                                ] + '-' * len(x_fasta[y].seq
                                             )
    else:
        for y in x_fasta:
            seq_coll[inv_map[y[0: 5]
                            ]
                    ] = seq_coll[inv_map[y[0: 5]
                                        ]
                                ] + str(x_fasta[y].seq)

# for z in seq_coll:
#     to_write.append(SeqRecord(id = z, 
#                               name = z,
#                               description = "",
#                               dbxrefs=[],
#                               seq = Seq(seq_coll[z])
#                              )
#                    )
# with open(final_core_genome_path, "w") as output_handle:
#         SeqIO.write(to_write, output_handle, "fasta")
        
print(time.asctime())

Fri Feb 10 12:35:35 2023
Fri Feb 10 12:36:34 2023


## pt make

In [8]:
conv_d = {'A': [1., 0., 0., 0.],
          'T': [0., 1., 0., 0.],
          'C': [0., 0., 1., 0.],
          'G': [0., 0., 0., 1.],
          'Y': [0., 0.5, 0.5, 0.], # T, C
          'R': [0.5, 0., 0., 0.5], # A, G
          'S': [0., 0., 0.5, 0.5], # C, G
          'W': [0.5, 0.5, 0., 0.], # A, T
          'K': [0., 0.5, 0., 0.5], # G, T
          'M': [0.5, 0., 0.5, 0.], # A, C
          'B': [0., 1/3, 1/3, 1/3], # T, C, G
          'D': [1/3, 1/3, 0., 1/3], # A, T, G
          'H': [1/3, 1/3, 1/3, 0.], # A, T, C
          'V': [1/3, 0., 1/3, 1/3], # A, C, G
          'N': [0.25, 0.25, 0.25, 0.25],
          '-': [0., 0., 0., 0.]
         }

In [9]:
pt_path = '/data/docker_qiime2_share_container_D/to_github/pt/CoreDL/pt/np/'

def one_hot_encoding(sequence, conv_d_):
    
    seq_tensor = [conv_d_[base] for base in sequence]
    seq_tensor = torch.tensor(seq_tensor).t()
    
    return seq_tensor

def DNA_to_pt(core_genomes, in_lst, convert_dict):
    
    for f in in_lst:
#         s_seq = str(core_genomes[f].seq)
        s_seq = str(core_genomes[f])
        
        b_seq = one_hot_encoding(s_seq, convert_dict)
        
        torch.save(b_seq, 
                   pt_path
                   + f 
                   + '.pt')

In [10]:
start_time_external = time.time()
print(time.asctime())

if __name__ == '__main__':
    procc_num = 8
    
    mission_lst = np.array_split(list(seq_coll.keys()), 
                                 procc_num
                                )
    
    jobs = []
    
    for x in range(0, procc_num):
        p = Process(target = DNA_to_pt,
                    args = (seq_coll, 
                            mission_lst[x], 
                            conv_d
                           )
                   )
        p.start()
        jobs.append(p)
        
    
    for z in jobs:
        z.join()
        
end_time_external = time.time()
print("final time usage " + str(end_time_external - start_time_external))
print(time.asctime())

Fri Feb 10 12:36:34 2023
final time usage 44.883763551712036
Fri Feb 10 12:37:18 2023
